# Practical Data Transformation and Analysis with Pandas

# Speaker

* Zong-han, Xie

* Majored in physics

* Previously a C++ developer developing simulation software for LCD.

* Currently working for Micron Memory Taiwan, building home-made B.I. system.

* Email: icbm0926@gmail.com

# Sequence
* Running complex SQL code on a SQL server creates heavy loading to the server.

* It is hard to write omplex analysis into a SQL code, you might want to write those code in Python.

* Intended fo people have built a B.I. system on top of Python enviroment ex: Django, flask, Pandas...etc.

* Who wants to exploit the power of Python and transfer their reporting services from SQL-related services (ex: SSRS) to Python.

* This talk mainly focues on the components related to split-apply-combine strategy which is main method for transforming and analyzing tabular data.

# Outline
* Basic Data Structures: 
  - Create Pandas DataFrame
  - Read our demo data
  - Indexing in DataFrame and Pandas Series
  - Assinging data into DataFrame and SettingsWithCopy Warning
* Text Handling with Pandas
  - Using "str" attributes to handle string and using regexp with it
* Merging and Concatenating tables
  - Concept of merging two tables (inner join, left/right join, outer join)
  - Concatenating tables
* Split-Apply-Combine strategy
  - Process Flow
  - GroupBy object
  - GroupBy.transform
  - GroupBy.apply
  - GroupBy.aggregate
* A small example

# Creating a DataFrame

In [9]:
import pandas as pd
pd.DataFrame({'Column_A': ['A1', 'A2', 'A3']
              , 'Column_B': ['A1', 'A2', 'A3']}
             , index=['a', 'b', 'c'])

,Column_A,Column_B
a,A1,A1
b,A2,A2
c,A3,A3


# Create a DataFrame from numpy array

In [10]:
import numpy as np
np_array = np.array(np.random.random((5,2)))
pd.DataFrame(np_array, columns=['Column_A', 'Column_B'])

,Column_A,Column_B
0,0.532799,0.515631
1,0.977685,0.476983
2,0.821461,0.488186
3,0.611671,0.793174
4,0.470772,0.520843


# Create a new column for DataFrame

In [11]:
import numpy as np
np_array = np.array(np.random.random((5,2)))
df = pd.DataFrame(np_array, columns=['Column_A', 'Column_B'])
df['column_C'] = pd.Series([1,2,3,4,5])
print(df)

   Column_A  Column_B  column_C
0  0.451639  0.110256         1
1  0.163979  0.829561         2
2  0.718608  0.981952         3
3  0.171740  0.743521         4
4  0.531219  0.759881         5


# Create DataFrame from a file
read in the demo data, data is from MovieLens, ml-1m dataset.

In [12]:
users_df = pd.read_csv("./ml-1m/users.dat"
                    , delimiter='::'
                    , header=None
                    , names=["UserID", "Gender", "Age", "Occupation", "Zip-code"]
                    , engine='python')
users_df.loc[0:3, :]

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460


In [13]:
movies_org_df = pd.read_csv("./ml-1m/movies.dat"
                            , sep='::'
                            , header=None
                            , names=["MovieID", "Title", "Genres"]
                            , engine='python')
movies_org_df.loc[0:3, :]

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama


In [14]:
import datetime as dt
ratings_df = pd.read_csv("./ml-1m/ratings.dat"
                         , sep='::'
                         , header=None
                         , names=["UserID", "MovieID", "Rating", "Timestamp"], engine='python')
ratings_df['rating_dt'] = pd.to_datetime(ratings_df['Timestamp'],unit='s')
ratings_df.loc[0:3, :]

,UserID,MovieID,Rating,Timestamp,rating_dt
0,1,1193,5,978300760,2000-12-31 22:12:40
1,1,661,3,978302109,2000-12-31 22:35:09
2,1,914,3,978301968,2000-12-31 22:32:48
3,1,3408,4,978300275,2000-12-31 22:04:35


# DataFrame column names

In [19]:
print(users_df.columns)
print(movies_org_df.columns)
print(ratings_df.columns)

Index(['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], dtype='object')
Index(['MovieID', 'Title', 'Genres'], dtype='object')
Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'rating_dt'], dtype='object')


# DataFrame row index

In [24]:
print(users_df.index)
print(list(users_df.index)[0:20])

RangeIndex(start=0, stop=6040, step=1)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


# DataFrame Indexing

In [15]:
movies_org_df['Title'][0]

'Toy Story (1995)'

In [25]:
movies_org_df['Title']

0                                        Toy Story (1995)
1                                          Jumanji (1995)
2                                 Grumpier Old Men (1995)
3                                Waiting to Exhale (1995)
4                      Father of the Bride Part II (1995)
5                                             Heat (1995)
6                                          Sabrina (1995)
7                                     Tom and Huck (1995)
8                                     Sudden Death (1995)
9                                        GoldenEye (1995)
10                         American President, The (1995)
11                     Dracula: Dead and Loving It (1995)
12                                           Balto (1995)
13                                           Nixon (1995)
14                                Cutthroat Island (1995)
15                                          Casino (1995)
16                           Sense and Sensibility (1995)
17            

In [8]:
# Make sure that you input a "list" of column names
users_df[['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']]

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [26]:
# This is frequently happened typo
users_df['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

KeyError: ('UserID', 'Gender', 'Age', 'Occupation', 'Zip-code')

In [27]:
# One can choose a index by treating it like an attribute of this DataFrame
movies_org_df.Title

0                                        Toy Story (1995)
1                                          Jumanji (1995)
2                                 Grumpier Old Men (1995)
3                                Waiting to Exhale (1995)
4                      Father of the Bride Part II (1995)
5                                             Heat (1995)
6                                          Sabrina (1995)
7                                     Tom and Huck (1995)
8                                     Sudden Death (1995)
9                                        GoldenEye (1995)
10                         American President, The (1995)
11                     Dracula: Dead and Loving It (1995)
12                                           Balto (1995)
13                                           Nixon (1995)
14                                Cutthroat Island (1995)
15                                          Casino (1995)
16                           Sense and Sensibility (1995)
17            

In [28]:
movies_org_df.Title[0]

'Toy Story (1995)'

In [17]:
movies_org_df.Title.loc[0]

'Toy Story (1995)'

In [18]:
movies_org_df.Title.iloc[0]

'Toy Story (1995)'

In [9]:
print(users_df['Occupation'][0:10])

0    10
1    16
2    15
3     7
4    20
5     9
6     1
7    12
8    17
9     1
Name: Occupation, dtype: int64


# Label-base vs. location base indexing
Get data row through its label index or its position inside DataFrame

In [49]:
# Through the position of rows in DataFrame
df = ratings_df.sort_values(by="rating_dt", ascending=False)
df.iloc[0:10, :]

,UserID,MovieID,Rating,Timestamp,rating_dt
825603,4958,1924,4,1046454590,2003-02-28 17:49:50
825724,4958,3264,4,1046454548,2003-02-28 17:49:08
825731,4958,2634,3,1046454548,2003-02-28 17:49:08
825438,4958,1407,5,1046454443,2003-02-28 17:47:23
825793,4958,2399,1,1046454338,2003-02-28 17:45:38
825526,4958,3489,4,1046454320,2003-02-28 17:45:20
825877,4958,2043,1,1046454282,2003-02-28 17:44:42
825497,4958,2453,4,1046454260,2003-02-28 17:44:20
879288,5312,3267,4,1046444711,2003-02-28 15:05:11
983967,5948,3098,4,1046437932,2003-02-28 13:12:12


In [50]:
# Through the position of rows in DataFrame
df = ratings_df.sort_values(by="rating_dt", ascending=False)
df.iloc[0:10, [0,1,2,3]]

,UserID,MovieID,Rating,Timestamp
825603,4958,1924,4,1046454590
825724,4958,3264,4,1046454548
825731,4958,2634,3,1046454548
825438,4958,1407,5,1046454443
825793,4958,2399,1,1046454338
825526,4958,3489,4,1046454320
825877,4958,2043,1,1046454282
825497,4958,2453,4,1046454260
879288,5312,3267,4,1046444711
983967,5948,3098,4,1046437932


In [52]:
# through DataFrame index
df = ratings_df.sort_values(by="rating_dt", ascending=False)
df.loc[0:100, :]

,UserID,MovieID,Rating,Timestamp,rating_dt
0,1,1193,5,978300760,2000-12-31 22:12:40
21,1,720,3,978300760,2000-12-31 22:12:40
44,1,260,4,978300760,2000-12-31 22:12:40
7,1,2804,5,978300719,2000-12-31 22:11:59
47,1,1207,4,978300719,2000-12-31 22:11:59
3,1,3408,4,978300275,2000-12-31 22:04:35
2493,19,1090,3,978300207,2000-12-31 22:03:27
2399,19,2841,5,978300188,2000-12-31 22:03:08
66,2,1687,3,978300174,2000-12-31 22:02:54
87,2,1544,4,978300174,2000-12-31 22:02:54


## SQL where statement equivalence
SQL: 
```SQL=
select * 
from ratings_df
where Rating < 3
```
Pandas:

In [45]:
ratings_df[ratings_df.Rating < 3].iloc[0:10, :]

,UserID,MovieID,Rating,Timestamp,rating_dt
67,2,1213,2,978298458,2000-12-31 21:34:18
73,2,434,2,978300174,2000-12-31 22:02:54
75,2,3107,2,978300002,2000-12-31 22:00:02
83,2,902,2,978298905,2000-12-31 21:41:45
91,2,3256,2,978299839,2000-12-31 21:57:19
114,2,3699,2,978299173,2000-12-31 21:46:13
125,2,2427,2,978299913,2000-12-31 21:58:33
133,2,95,2,978300143,2000-12-31 22:02:23
148,2,21,1,978299839,2000-12-31 21:57:19
151,2,1090,2,978298580,2000-12-31 21:36:20


SQL: 
```SQL=
select * 
from ratings_df
where rating_dt < '2003-02-01' and rating_dt >= '2003-01-01'
```
Pandas:

In [62]:
import datetime as dt
df = ratings_df[(ratings_df.rating_dt 
                 < dt.datetime(2003, 2, 1)) &
        (ratings_df.rating_dt >= dt.datetime(2003, 1, 1))]
print(df[df.rating_dt == df.rating_dt.max()])
print("--------------------------------------------------------------------")
print(df[df.rating_dt == df.rating_dt.min()])

        UserID  MovieID  Rating   Timestamp           rating_dt
329894    1946      144       4  1044033786 2003-01-31 17:23:06
329916    1946     1220       4  1044033786 2003-01-31 17:23:06
--------------------------------------------------------------------
        UserID  MovieID  Rating   Timestamp           rating_dt
172198    1088     3900       3  1041384785 2003-01-01 01:33:05


## Deprecated indexing method

In [18]:
users_df['Occupation'].ix[0:10]

0     10
1     16
2     15
3      7
4     20
5      9
6      1
7     12
8     17
9      1
10     1
Name: Occupation, dtype: int64

<font color='#c94c4c'> Indexing with .ix attribute has beedn deprecated since Pandas 0.20.0 </font>

<pre>.ix offers a lot of magic on the inference of what the user wants to do. To wit, .ix can decide to index positionally OR via labels depending on the data type of the index. This has caused quite a bit of user confusion over the years. 

The recommended methods of indexing are:

.loc if you want to label index
.iloc if you want to positionally index.
</pre>

# Change data in DataFrame

In [43]:
copy_users_df = users_df.copy()
copy_users_df = copy_users_df.iloc[0:5, :]
print(copy_users_df['Gender'])
copy_users_df['Gender'][2] = 'F'
print(copy_users_df['Gender'])

0    F
1    M
2    M
3    M
4    M
5    F
Name: Gender, dtype: object
0    F
1    M
2    F
3    M
4    M
5    F
Name: Gender, dtype: object


## Following assignment doesn't work
It gives you "SettingWithCopyWarning"

In [47]:
copy_users_df = users_df.copy()
copy_users_df[copy_users_df.UserID < 10]['Gender'] = 'F'
print(copy_users_df[copy_users_df.UserID < 10])

   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455
5       6      F   50           9    55117
6       7      M   35           1    06810
7       8      M   25          12    11413
8       9      M   25          17    61614


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


# SettingsWithCopy Warning and Chained Indexing
* http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

* If one is using chained indexing in assignment statement, there is an extra __getitem__ funtion call under the hood.

* __getitem__ returns a copy or a view depending on memory layout of the array, about which pandas makes no guarantees.

* To successfully changing data in the DataFrame, using "loc" or "iloc" to avoid chained indexing.

# Text Handling

In [18]:
movies_org_df.ix[0:10]

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


# Split Genres and put into rows

In [33]:
rows = []
for _, row in movies_org_df.iterrows():
    for gen in row.Genres.split('|'):
        rows.append([row['MovieID'], row['Title'], gen])
movies_df = pd.DataFrame(rows, columns=movies_org_df.columns)

# Merge Data and Concat Data

* Pandas.merge and Pandas.concat

* Pandas.merge is analog to join in SQL (inner, left, rught, outer join)

* Pandas.concat is analog to "union all" in SQL.

# Merge the demo data together

In [36]:
merged_df = ratings_df.merge(users_df
                             , on=['UserID']
                             , how='left') \
                      .merge(movies_df
                             , on=['MovieID']
                             , how='left')
merged_df.ix[0:10]

,UserID,MovieID,Rating,Timestamp,rating_dt,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,2000-12-31 22:12:40,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,2000-12-31 22:35:09,F,1,10,48067,James and the Giant Peach (1996),Animation
2,1,661,3,978302109,2000-12-31 22:35:09,F,1,10,48067,James and the Giant Peach (1996),Children's
3,1,661,3,978302109,2000-12-31 22:35:09,F,1,10,48067,James and the Giant Peach (1996),Musical
4,1,914,3,978301968,2000-12-31 22:32:48,F,1,10,48067,My Fair Lady (1964),Musical
5,1,914,3,978301968,2000-12-31 22:32:48,F,1,10,48067,My Fair Lady (1964),Romance
6,1,3408,4,978300275,2000-12-31 22:04:35,F,1,10,48067,Erin Brockovich (2000),Drama
7,1,2355,5,978824291,2001-01-06 23:38:11,F,1,10,48067,"Bug's Life, A (1998)",Animation
8,1,2355,5,978824291,2001-01-06 23:38:11,F,1,10,48067,"Bug's Life, A (1998)",Children's
9,1,2355,5,978824291,2001-01-06 23:38:11,F,1,10,48067,"Bug's Life, A (1998)",Comedy


# Split - Apply - Combine Strategy

The basic cooncept of split-apply-combine strategy 

* Split
    - Split original data into groups

* Apply
    - Apply functions to data within each group independently

* Combine
    - Merge results into a data structure

# Data Aggregation

```
select 
...
AVG(Ratings.Rating) as "avg_ratings"
...
from Ratings
group by Ratings.MovieId
```

```
df.assign(avg_ratings = ratings_df.groupby("MovieID")['Rating'].transform(np.mean))
```

* Data aggregation in Pandas uses GroupBy.apply, GroupBy.transform and GroupBy.aggregate.

# GroupBy objects

In [8]:
merged_df.groupby(["Occupation", "Genres"])

# See what's in the GroupBy objects

In [9]:
grouped_ratings = merged_df.groupby(["Occupation", "Genres"])
for key, group_df in grouped_ratings:
    print("group keys: " + str(key))
    print(group_df.iloc[:5])
    break

group keys: ('K-12 student', 'Action')
     UserID  MovieID  Rating  Timestamp           rating_dt Gender  Age  \
10        1     1197       3  978302268 2000-12-31 22:37:48      F    1   
14        1     1287       5  978302039 2000-12-31 22:33:59      F    1   
90        1     2692       4  978301570 2000-12-31 22:26:10      F    1   
93        1      260       4  978300760 2000-12-31 22:12:40      F    1   
104       1     2028       5  978301619 2000-12-31 22:26:59      F    1   

    Zip-code    Occupation                                      Title  Genres  
10     48067  K-12 student                 Princess Bride, The (1987)  Action  
14     48067  K-12 student                             Ben-Hur (1959)  Action  
90     48067  K-12 student           Run Lola Run (Lola rennt) (1998)  Action  
93     48067  K-12 student  Star Wars: Episode IV - A New Hope (1977)  Action  
104    48067  K-12 student                 Saving Private Ryan (1998)  Action  


# Let's do Split-Apply-Combine manually

In [10]:
results = {'Occupation': [], 'Genres': [], 'Rating_mean':[]}
grouped_ratings = merged_df.groupby(["Occupation", "Genres"])  # split
for key, group_df in grouped_ratings:
    results['Occupation'].append(key[0])
    results['Genres'].append(key[1])
    results['Rating_mean'].append(group_df.Rating.mean())  # apply
pd.DataFrame(results).ix[0:10]  # combine

,Genres,Occupation,Rating_mean
0,Action,K-12 student,3.497116
1,Adventure,K-12 student,3.425658
2,Animation,K-12 student,3.463956
3,Children's,K-12 student,3.220679
4,Comedy,K-12 student,3.497200
5,Crime,K-12 student,3.687085
6,Documentary,K-12 student,3.581633
7,Drama,K-12 student,3.782167
8,Fantasy,K-12 student,3.298039
9,Film-Noir,K-12 student,4.212766


In [30]:
import numpy as np
tmp = merged_df[merged_df.Occupation == 'K-12 student'].copy()
tmp.loc[:,'Rating_mean'] = tmp.groupby(["Occupation", "Genres"])['Rating'].transform(np.mean)
print(tmp[['Occupation', 'Genres', 'Rating_mean']].sort_values(by='Genres').iloc[1:10])

           Occupation  Genres  Rating_mean
804564   K-12 student  Action     3.497116
363454   K-12 student  Action     3.497116
1806560  K-12 student  Action     3.497116
772492   K-12 student  Action     3.497116
1806556  K-12 student  Action     3.497116
659674   K-12 student  Action     3.497116
706974   K-12 student  Action     3.497116
827117   K-12 student  Action     3.497116
363460   K-12 student  Action     3.497116


** GroupBy.transform() returns a Pandas Series with the same index as those in original DataFrame **

** Therefore, it's easy to combine data back to the original data. **

In [12]:
print(tmp[['Occupation', 'Genres', 'Rating_mean']].sort_values(by='Genres').drop_duplicates().iloc[0:10])

           Occupation       Genres  Rating_mean
418625   K-12 student       Action     3.497116
831363   K-12 student    Adventure     3.425658
136334   K-12 student    Animation     3.463956
453973   K-12 student   Children's     3.220679
789324   K-12 student       Comedy     3.497200
692103   K-12 student        Crime     3.687085
1931149  K-12 student  Documentary     3.581633
790387   K-12 student        Drama     3.782167
521989   K-12 student      Fantasy     3.298039
364059   K-12 student    Film-Noir     4.212766


In [13]:
merged_df.groupby(['Occupation', 'Genres'])['Rating'].mean()

Occupation         Genres     
K-12 student       Action         3.497116
                   Adventure      3.425658
                   Animation      3.463956
                   Children's     3.220679
                   Comedy         3.497200
                   Crime          3.687085
                   Documentary    3.581633
                   Drama          3.782167
                   Fantasy        3.298039
                   Film-Noir      4.212766
                   Horror         3.237795
                   Musical        3.556738
                   Mystery        3.636612
                   Romance        3.624415
                   Sci-Fi         3.443795
                   Thriller       3.554131
                   War            3.880144
                   Western        3.513333
academic/educator  Action         3.392063
                   Adventure      3.424278
                   Animation      3.693399
                   Children's     3.459286
                   Come

In [14]:
merged_df.groupby(['Occupation', 'Genres'])['Rating'].agg(np.mean).ix['K-12 student']

Genres
Action         3.497116
Adventure      3.425658
Animation      3.463956
Children's     3.220679
Comedy         3.497200
Crime          3.687085
Documentary    3.581633
Drama          3.782167
Fantasy        3.298039
Film-Noir      4.212766
Horror         3.237795
Musical        3.556738
Mystery        3.636612
Romance        3.624415
Sci-Fi         3.443795
Thriller       3.554131
War            3.880144
Western        3.513333
Name: Rating, dtype: float64

In [15]:
merged_df.groupby(['Occupation', 'Genres'])['Rating'].agg(np.mean).reset_index().ix[0:5]

,Occupation,Genres,Rating
0,K-12 student,Action,3.497116
1,K-12 student,Adventure,3.425658
2,K-12 student,Animation,3.463956
3,K-12 student,Children's,3.220679
4,K-12 student,Comedy,3.497200
5,K-12 student,Crime,3.687085


In [27]:
merged_df.groupby(['Occupation', 'Genres']).agg({'Rating': np.mean}).reset_index().ix[0:5]

,Occupation,Genres,Rating
0,K-12 student,Action,3.497116
1,K-12 student,Adventure,3.425658
2,K-12 student,Animation,3.463956
3,K-12 student,Children's,3.220679
4,K-12 student,Comedy,3.497200
5,K-12 student,Crime,3.687085


In [44]:
#merged_df.groupby(['Occupation', 'Genres']).agg({'Rating': 'quantile(0.25)'}).reset_index()
merged_df.groupby(['Occupation', 'Genres']).agg({'Rating': 'quantile(0.25)'}).reset_index()

AttributeError: 'SeriesGroupBy' object has no attribute 'quantile(0.25)'

# A simple example